In [4]:
import os
import json
import pandas as pd

# Columns in desired final order
FINAL_COLUMNS = [
    'hospital_name', 'street_address', 'city', 'state', 'zip_code',
    'description', 'billing_code', 'billing_code_type', 'standard_charge',
    'discounted_cash_charge', 'payer_name', 'plan_name',
    'negotiated_dollar', 'negotiated_percentage', 'estimated_amount',
    'min_charge', 'max_charge'
]

def extract_cpt_with_payer_data(filepath):
    with open(filepath, 'r', encoding='latin-1') as f:
        data = json.load(f)

    # Extract hospital and address info
    hospital = data.get("hospital_name", "")
    addr = data.get("hospital_address", [""])[0].split(',')
    street = addr[0].strip() if len(addr) > 0 else ""
    city = addr[1].strip() if len(addr) > 1 else ""
    state_zip = addr[2].strip().split() if len(addr) > 2 else ["", ""]
    state = state_zip[0]
    zip_code = state_zip[1] if len(state_zip) > 1 else ""

    rows = []

    for item in data.get("standard_charge_information", []):
        code_info = item.get("code_information", [{}])[0]
        if code_info.get("type", "").upper() != "CPT":
            continue

        billing_code = code_info.get("code", "")
        description = item.get("description", "")

        for charge in item.get("standard_charges", []):
            base = {
                "hospital_name": hospital,
                "street_address": street,
                "city": city,
                "state": state,
                "zip_code": zip_code,
                "description": description,
                "billing_code": billing_code,
                "billing_code_type": "CPT",
                "standard_charge": charge.get("gross_charge"),
                "discounted_cash_charge": charge.get("discounted_cash"),
                "negotiated_percentage": "",
                "estimated_amount": "",
                "min_charge": charge.get("minimum", ""),
                "max_charge": charge.get("maximum", "")
            }

            for payer in charge.get("payers_information", [{}]):
                rows.append({
                    **base,
                    "payer_name": payer.get("payer_name", ""),
                    "plan_name": payer.get("plan_name", ""),
                    "negotiated_dollar": payer.get("standard_charge_dollar", "")
                })

    return pd.DataFrame(rows)

def batch_process_jsons(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            filepath = os.path.join(input_folder, filename)
            print(f"📄 Processing: {filename}")
            df = extract_cpt_with_payer_data(filepath)

            if not df.empty:
                df = df[FINAL_COLUMNS]  # Ensure column order
                hospital_name = os.path.splitext(filename)[0]
                output_path = os.path.join(output_folder, f"{hospital_name}_CPT_cleaned.csv")
                df.to_csv(output_path, index=False)
                print(f"✅ Saved: {output_path}")
            else:
                print(f"⚠️ No CPT records found in {filename}")

# === USAGE ===
batch_process_jsons(
    input_folder= '/Users/unmonadas/Desktop/ALL PROJECT FILES/New/JSON CA',
    output_folder='/Users/unmonadas/Desktop/Cleaned_Files_CA_JSON'
)


📄 Processing: 941439787_sierra-nevada-memorial-hospital_standardcharges.json
✅ Saved: /Users/unmonadas/Desktop/Cleaned_Files_CA_JSON/941439787_sierra-nevada-memorial-hospital_standardcharges_CPT_cleaned.csv
📄 Processing: 941196203_mercy-hospital-of-folsom_standardcharges.json
✅ Saved: /Users/unmonadas/Desktop/Cleaned_Files_CA_JSON/941196203_mercy-hospital-of-folsom_standardcharges_CPT_cleaned.csv
📄 Processing: 941196203_mercy-general-hospital_standardcharges.json
✅ Saved: /Users/unmonadas/Desktop/Cleaned_Files_CA_JSON/941196203_mercy-general-hospital_standardcharges_CPT_cleaned.csv
📄 Processing: 941196203_mercy-san-juan-medical-center_standardcharges.json
✅ Saved: /Users/unmonadas/Desktop/Cleaned_Files_CA_JSON/941196203_mercy-san-juan-medical-center_standardcharges_CPT_cleaned.csv
📄 Processing: 815009488_methodist-hospital-of-sacramento_standardcharges.json
✅ Saved: /Users/unmonadas/Desktop/Cleaned_Files_CA_JSON/815009488_methodist-hospital-of-sacramento_standardcharges_CPT_cleaned.csv